In [1]:
import QuantLib as ql

In [2]:
refDate = ql.Date(30,4,2014)

In [3]:
ql.Settings.instance().evaluationDate=refDate

In [4]:
print("EvalDate: ",ql.Settings.instance().evaluationDate)

EvalDate:  April 30th, 2014


In [5]:
forward6mLevel = 0.025
oisLevel = 0.02

In [6]:
forward6mQuote = ql.QuoteHandle(ql.SimpleQuote(forward6mLevel))
oisQuote = ql.QuoteHandle(ql.SimpleQuote(oisLevel))

In [7]:
yts6m = ql.YieldTermStructureHandle(ql.FlatForward(0, ql.TARGET(), forward6mQuote, ql.Actual365Fixed() ))
ytsOis = ql.YieldTermStructureHandle(ql.FlatForward(0, ql.TARGET(), oisQuote, ql.Actual365Fixed() ))

In [8]:
euribor6m = ql.Euribor6M(yts6m)

In [9]:
volLevel = 0.20

In [10]:
volQuote = ql.QuoteHandle(ql.SimpleQuote(volLevel))

In [11]:
swaptionVol = ql.ConstantSwaptionVolatility(0, ql.TARGET(), ql.ModifiedFollowing, volQuote,
                                                                                ql.Actual365Fixed())

In [12]:
swaptionVolHandle = ql.SwaptionVolatilityStructureHandle(ql.ConstantSwaptionVolatility(0, ql.TARGET(), ql.ModifiedFollowing, volQuote,
                                                                                ql.Actual365Fixed()))

In [13]:
strike = 0.04

In [14]:
effectiveDate = ql.TARGET().advance(refDate, ql.Period(2,ql.Days))

In [15]:
effectiveDate

Date(5,5,2014)

In [16]:
maturityDate = ql.TARGET().advance(effectiveDate, ql.Period(10,ql.Years))

In [17]:
maturityDate

Date(6,5,2024)

In [18]:
fixedSchedule = ql.Schedule(effectiveDate, maturityDate, ql.Period(1,ql.Years), ql.TARGET(),
                            ql.ModifiedFollowing, ql.ModifiedFollowing,
                            ql.DateGeneration.Forward, False)

In [19]:
for i in fixedSchedule:
    print(i)

May 5th, 2014
May 5th, 2015
May 5th, 2016
May 5th, 2017
May 7th, 2018
May 6th, 2019
May 5th, 2020
May 5th, 2021
May 5th, 2022
May 5th, 2023
May 6th, 2024


In [20]:
floatingSchedule = ql.Schedule(effectiveDate, maturityDate, ql.Period(6,ql.Months), ql.TARGET(),
                            ql.ModifiedFollowing, ql.ModifiedFollowing,
                            ql.DateGeneration.Forward, False)

In [21]:
for i in floatingSchedule:
    print(i)

May 5th, 2014
November 5th, 2014
May 5th, 2015
November 5th, 2015
May 5th, 2016
November 7th, 2016
May 5th, 2017
November 6th, 2017
May 7th, 2018
November 5th, 2018
May 6th, 2019
November 5th, 2019
May 5th, 2020
November 5th, 2020
May 5th, 2021
November 5th, 2021
May 5th, 2022
November 7th, 2022
May 5th, 2023
November 6th, 2023
May 6th, 2024


In [22]:
underlying = ql.NonstandardSwap(ql.VanillaSwap.Payer, [1.0]*(len(fixedSchedule)-1), [1.0]*(len(floatingSchedule)-1), fixedSchedule,
                                              [strike]*(len(fixedSchedule)-1), ql.Thirty360(), floatingSchedule, 
                                              euribor6m, [1.0]*(len(floatingSchedule)-1), [0.0]*(len(floatingSchedule)-1), ql.Actual360())

In [23]:
for cf in underlying.fixedLeg():
    print (cf.date(), cf.amount())

May 5th, 2015 0.040000000000000036
May 5th, 2016 0.040000000000000036
May 5th, 2017 0.040000000000000036
May 7th, 2018 0.04022222222222216
May 6th, 2019 0.03988888888888886
May 5th, 2020 0.03988888888888886
May 5th, 2021 0.040000000000000036
May 5th, 2022 0.040000000000000036
May 5th, 2023 0.040000000000000036
May 6th, 2024 0.04011111111111121


In [24]:
for cf in underlying.floatingLeg():
    print (cf.date(), cf.amount())

November 5th, 2014 0.012682488917620917
May 5th, 2015 0.012474424851870225
November 5th, 2015 0.012682488917621138
May 5th, 2016 0.012543774790186868
November 7th, 2016 0.012821222048292169
May 5th, 2017 0.012335739224410558
November 6th, 2017 0.012751853107389575
May 7th, 2018 0.012543774790186868
November 5th, 2018 0.012543774790186868
May 6th, 2019 0.012543774790186868
November 5th, 2019 0.012613129478662223
May 5th, 2020 0.012543774790186868
November 5th, 2020 0.012682488917621138
May 5th, 2021 0.012474424851870227
November 5th, 2021 0.01268248891762114
May 5th, 2022 0.012474424851870003
November 7th, 2022 0.012821222048292391
May 5th, 2023 0.012335739224410556
November 6th, 2023 0.012751853107389573
May 6th, 2024 0.01254377479018687


In [25]:
exerciseDates = [ql.TARGET().advance(fixedSchedule[i], ql.Period(-2, ql.Days)) for i in range(1,10)]

In [26]:
exerciseDates

[Date(30,4,2015),
 Date(3,5,2016),
 Date(3,5,2017),
 Date(3,5,2018),
 Date(2,5,2019),
 Date(30,4,2020),
 Date(3,5,2021),
 Date(3,5,2022),
 Date(3,5,2023)]

In [27]:
exercise = ql.BermudanExercise(exerciseDates, False)

In [28]:
swaption = ql.NonstandardSwaption(underlying, exercise)

In [29]:
stepDates = [exerciseDates[i] for i in range(0, len(exerciseDates)-1)]

In [30]:
sigmas = [ql.QuoteHandle(ql.SimpleQuote(0.01)) for i in range(0, len(stepDates)+1)]

In [31]:
reversion = 0.01

In [32]:
reversions = [ql.QuoteHandle(ql.SimpleQuote(reversion))]

In [33]:
gsr = ql.Gsr(yts6m, stepDates, sigmas, reversions)

In [34]:
swaptionEngine = ql.Gaussian1dSwaptionEngine(gsr, 64, 7.0, True, False, ytsOis)

In [35]:
nonstandardSwaptionEngine = ql.Gaussian1dNonstandardSwaptionEngine(gsr, 64, 7.0, True, False, ql.QuoteHandle(),ytsOis)

In [36]:
swaption.setPricingEngine(nonstandardSwaptionEngine)

In [37]:
swapBase = ql.EuriborSwapIsdaFixA(ql.Period(10,ql.Years), yts6m, ytsOis)

In [38]:
basket = swaption.calibrationBasket(swapBase, swaptionVol, "Naive")

In [39]:
def printBasket(basket):
    print()
    print('{:<20}{:<20}{:<20}{:<14}{:<12}{:<14}'.format("Expiry", "Maturity", "Nominal", "Rate", "Pay/Rec", "Market ivol"))
    print("===================="*4+"==================")
    for helper in basket:
        endDate = helper.swaptionMaturityDate()
        nominal = helper.swaptionNominal()
        vol = helper.volatility().value()
        rate = helper.swaptionStrike()
        expiry = helper.swaptionExpiryDate()
        swp_type = 0 # to be SWIGed
        print('{:<20}{:<20}{:<20}{:<14}{:<12}{:<14}'.format(str(expiry), str(endDate), ("%.2f"%nominal), ("%.6f"%rate), str(swp_type), ("%.6f"%vol)))

In [40]:
printBasket(basket)


Expiry              Maturity            Nominal             Rate          Pay/Rec     Market ivol   
April 30th, 2015    May 6th, 2024       1.00                0.025307      0           0.200000      
May 3rd, 2016       May 6th, 2024       1.00                0.025300      0           0.200000      
May 3rd, 2017       May 6th, 2024       1.00                0.025303      0           0.200000      
May 3rd, 2018       May 6th, 2024       1.00                0.025306      0           0.200000      
May 2nd, 2019       May 6th, 2024       1.00                0.025311      0           0.200000      
April 30th, 2020    May 6th, 2024       1.00                0.025300      0           0.200000      
May 3rd, 2021       May 6th, 2024       1.00                0.025306      0           0.200000      
May 3rd, 2022       May 6th, 2024       1.00                0.025318      0           0.200000      
May 3rd, 2023       May 6th, 2024       1.00                0.025353      0           0.20

In [41]:
for helper in basket:
    helper.setPricingEngine(swaptionEngine)

In [42]:
method = ql.LevenbergMarquardt()

In [43]:
ec = ql.EndCriteria(1000, 10, 1e-8, 1e-8, 1e-8)

In [44]:
gsr.calibrateVolatilitiesIterative(basket, method, ec)

In [45]:
def printModelCalibration(basket, volatility):
    print()
    print('{:<20}{:<14}{:<20}{:<20}{:<14}{:<14}'.format("Expiry", "Model sigma", "Model price", "Market price", "Model ivol", "Market ivol"))
    print("===================="*5)
    for i in range(0,len(basket)):
        helper = basket[i]
        expiry = helper.swaptionExpiryDate()
        print('{:<20}{:<14}{:<20}{:<20}{:<14}{:<14}'.format(str(expiry), ("%.6f"%volatility[i]), ("%.6f"%helper.modelValue()),
                                                           ("%.6f"%helper.marketValue()), ("%.6f"%helper.impliedVolatility(helper.modelValue(), 1e-6, 1000, 0.0, 2.0)),
                                                           ("%.6f"%helper.volatility().value())))
    if (len(volatility) > len(basket)):
        print('{:<20}'.format(("%.6f"%volatility[-1])))
        

In [46]:
printModelCalibration(basket, gsr.volatility())


Expiry              Model sigma   Model price         Market price        Model ivol    Market ivol   
April 30th, 2015    0.005178      0.016111            0.016111            0.199999      0.200000      
May 3rd, 2016       0.005156      0.020062            0.020062            0.200000      0.200000      
May 3rd, 2017       0.005149      0.021229            0.021229            0.200000      0.200000      
May 3rd, 2018       0.005129      0.020738            0.020738            0.200000      0.200000      
May 2nd, 2019       0.005132      0.019096            0.019096            0.200000      0.200000      
April 30th, 2020    0.005074      0.016537            0.016537            0.200000      0.200000      
May 3rd, 2021       0.005091      0.013253            0.013253            0.200000      0.200000      
May 3rd, 2022       0.005097      0.009342            0.009342            0.200000      0.200000      
May 3rd, 2023       0.005001      0.004910            0.004910          

In [47]:
print("Bermudan swaption NPV (ATM calibrated GSR) = %.6f" % swaption.NPV())

Bermudan swaption NPV (ATM calibrated GSR) = 0.003808


In [48]:
basket = swaption.calibrationBasket(swapBase, swaptionVol, "MaturityStrikeByDeltaGamma")

In [49]:
printBasket(basket)


Expiry              Maturity            Nominal             Rate          Pay/Rec     Market ivol   
April 30th, 2015    May 6th, 2024       1.00                0.040000      0           0.200000      
May 3rd, 2016       May 6th, 2024       1.00                0.040000      0           0.200000      
May 3rd, 2017       May 6th, 2024       1.00                0.040000      0           0.200000      
May 3rd, 2018       May 7th, 2024       1.00                0.040000      0           0.200000      
May 2nd, 2019       May 6th, 2024       1.00                0.040000      0           0.200000      
April 30th, 2020    May 6th, 2024       1.00                0.040000      0           0.200000      
May 3rd, 2021       May 6th, 2024       1.00                0.040000      0           0.200000      
May 3rd, 2022       May 6th, 2024       1.00                0.040000      0           0.200000      
May 3rd, 2023       May 6th, 2024       1.00                0.040000      0           0.20

In [50]:
for helper in basket:
    helper.setPricingEngine(swaptionEngine)

In [51]:
gsr.calibrateVolatilitiesIterative(basket, method, ec)

In [52]:
printModelCalibration(basket, gsr.volatility())


Expiry              Model sigma   Model price         Market price        Model ivol    Market ivol   
April 30th, 2015    0.006508      0.000191            0.000191            0.200000      0.200000      
May 3rd, 2016       0.006502      0.001412            0.001412            0.200000      0.200000      
May 3rd, 2017       0.006480      0.002905            0.002905            0.200000      0.200000      
May 3rd, 2018       0.006464      0.004091            0.004091            0.200000      0.200000      
May 2nd, 2019       0.006422      0.004765            0.004765            0.200000      0.200000      
April 30th, 2020    0.006445      0.004869            0.004869            0.200000      0.200000      
May 3rd, 2021       0.006433      0.004433            0.004433            0.200000      0.200000      
May 3rd, 2022       0.006332      0.003454            0.003454            0.200000      0.200000      
May 3rd, 2023       0.006295      0.001973            0.001973          

In [53]:
print("Bermudan swaption NPV (deal strike calibrated GSR) = %.6f" % swaption.NPV())

Bermudan swaption NPV (deal strike calibrated GSR) = 0.007627


In [54]:
nominalFixed = [ 1.0 - float(i) / (len(fixedSchedule) - 1) for i in range(0,len(fixedSchedule)-1)]
nominalFloating = [ n for n in nominalFixed for repetitions in range(2)]
strikes = [strike]*len(nominalFixed)

In [55]:
underlying2 = ql.NonstandardSwap(ql.VanillaSwap.Payer, nominalFixed, nominalFloating, fixedSchedule,
                                              strikes, ql.Thirty360(), floatingSchedule, 
                                              euribor6m, [1.0]*(len(floatingSchedule)-1), [0.0]*(len(floatingSchedule)-1), ql.Actual360())

In [56]:
for cf in underlying2.fixedLeg():
    print (cf.date(), cf.amount())

May 5th, 2015 0.040000000000000036
May 5th, 2016 0.03600000000000003
May 5th, 2017 0.03200000000000003
May 7th, 2018 0.02815555555555551
May 6th, 2019 0.023933333333333317
May 5th, 2020 0.01994444444444443
May 5th, 2021 0.016000000000000014
May 5th, 2022 0.012000000000000012
May 5th, 2023 0.008000000000000005
May 6th, 2024 0.00401111111111112


In [57]:
swaption2 = ql.NonstandardSwaption(underlying2, exercise)

In [58]:
swaption2.setPricingEngine(nonstandardSwaptionEngine)

In [59]:
basket = swaption2.calibrationBasket(swapBase, swaptionVol, "MaturityStrikeByDeltaGamma")

In [60]:
printBasket(basket)


Expiry              Maturity            Nominal             Rate          Pay/Rec     Market ivol   
April 30th, 2015    August 5th, 2021    0.72                0.039997      0           0.200000      
May 3rd, 2016       December 6th, 2021  0.64                0.040003      0           0.200000      
May 3rd, 2017       May 5th, 2022       0.56                0.040005      0           0.200000      
May 3rd, 2018       September 7th, 2022 0.49                0.040004      0           0.200000      
May 2nd, 2019       January 6th, 2023   0.41                0.040008      0           0.200000      
April 30th, 2020    May 5th, 2023       0.33                0.039994      0           0.200000      
May 3rd, 2021       September 5th, 2023 0.26                0.039995      0           0.200000      
May 3rd, 2022       January 5th, 2024   0.18                0.040031      0           0.200000      
May 3rd, 2023       May 6th, 2024       0.10                0.040000      0           0.20

In [61]:
nominalFixed2 = [1.0]*len(nominalFixed)
nominalFloating2 = [0.0]*len(nominalFloating)

In [62]:
underlying3 = ql.NonstandardSwap(ql.VanillaSwap.Receiver, nominalFixed2, nominalFloating2, fixedSchedule,
                                              strikes, ql.Thirty360(), floatingSchedule, 
                                              euribor6m, [1.0]*(len(floatingSchedule)-1), [0.0]*(len(floatingSchedule)-1), ql.Actual360(),
                                False, True)

In [63]:
for cf in underlying3.fixedLeg():
    print (cf.date(), cf.amount())

May 5th, 2015 0.040000000000000036
May 5th, 2016 0.040000000000000036
May 5th, 2017 0.040000000000000036
May 7th, 2018 0.04022222222222216
May 6th, 2019 0.03988888888888886
May 5th, 2020 0.03988888888888886
May 5th, 2021 0.040000000000000036
May 5th, 2022 0.040000000000000036
May 5th, 2023 0.040000000000000036
May 6th, 2024 0.04011111111111121
May 6th, 2024 1.0


In [64]:
exercise2 = ql.RebatedExercise(exercise, [-1.0]*len(exerciseDates), 2, ql.TARGET())

In [65]:
swaption3 = ql.NonstandardSwaption(underlying3, exercise2)

In [66]:
oas0 = ql.SimpleQuote(0.0)
oas100 = ql.SimpleQuote(0.01)
oas = ql.RelinkableQuoteHandle(oas0)

In [67]:
nonstandardSwaptionEngine2 = ql.Gaussian1dNonstandardSwaptionEngine(gsr, 64, 7.0, True, False, oas) #change discounting to 6m

In [68]:
swaption3.setPricingEngine(nonstandardSwaptionEngine2)

In [69]:
basket = swaption3.calibrationBasket(swapBase, swaptionVol, "MaturityStrikeByDeltaGamma")

In [70]:
printBasket(basket)


Expiry              Maturity            Nominal             Rate          Pay/Rec     Market ivol   
April 30th, 2015    April 5th, 2024     0.98                0.039952      0           0.200000      
May 3rd, 2016       April 5th, 2024     0.99                0.039952      0           0.200000      
May 3rd, 2017       May 6th, 2024       0.99                0.039952      0           0.200000      
May 3rd, 2018       May 7th, 2024       0.99                0.039952      0           0.200000      
May 2nd, 2019       May 6th, 2024       0.99                0.039952      0           0.200000      
April 30th, 2020    May 6th, 2024       0.99                0.039951      0           0.200000      
May 3rd, 2021       May 6th, 2024       0.99                0.039951      0           0.200000      
May 3rd, 2022       May 6th, 2024       0.99                0.039952      0           0.200000      
May 3rd, 2023       May 6th, 2024       1.00                0.039949      0           0.20

In [71]:
for helper in basket:
    helper.setPricingEngine(swaptionEngine)

In [72]:
gsr.calibrateVolatilitiesIterative(basket, method, ec)

In [73]:
print("Bond's bermudan call right npv = %.6f" % swaption3.NPV())

Bond's bermudan call right npv = 0.115409


In [74]:
oas.linkTo(oas100) # Credit Spread 100bp

In [75]:
basket = swaption3.calibrationBasket(swapBase, swaptionVol, "MaturityStrikeByDeltaGamma")

In [76]:
printBasket(basket)


Expiry              Maturity            Nominal             Rate          Pay/Rec     Market ivol   
April 30th, 2015    February 5th, 2024  0.96                0.029608      0           0.200000      
May 3rd, 2016       March 5th, 2024     0.97                0.029605      0           0.200000      
May 3rd, 2017       April 5th, 2024     0.97                0.029608      0           0.200000      
May 3rd, 2018       April 8th, 2024     0.97                0.029610      0           0.200000      
May 2nd, 2019       April 8th, 2024     0.98                0.029608      0           0.200000      
April 30th, 2020    May 6th, 2024       0.98                0.029612      0           0.200000      
May 3rd, 2021       May 6th, 2024       0.99                0.029609      0           0.200000      
May 3rd, 2022       May 6th, 2024       0.99                0.029603      0           0.200000      
May 3rd, 2023       May 6th, 2024       1.00                0.029586      0           0.20

In [77]:
for helper in basket:
    helper.setPricingEngine(swaptionEngine)

In [78]:
gsr.calibrateVolatilitiesIterative(basket, method, ec)

In [79]:
print("Bond's bermudan call right npv (oas = 100bp) = %.6f" % swaption3.NPV())

Bond's bermudan call right npv (oas = 100bp) = 0.044980


In [80]:
underlying4 = ql.FloatFloatSwap(ql.VanillaSwap.Payer, [1.0]*(len(fixedSchedule)-1), [1.0]*(len(floatingSchedule)-1), fixedSchedule,
                               swapBase, ql.Thirty360(), floatingSchedule,
                               euribor6m, ql.Actual360(), False, False,
                               [1.0]*(len(fixedSchedule)-1),[0.0]*(len(fixedSchedule)-1),[] ,[] ,[1.0]*(len(floatingSchedule)-1),
                                [0.001]*(len(floatingSchedule)-1),[],[])

In [81]:
swaption4 = ql.FloatFloatSwaption(underlying4, exercise)

In [82]:
floatSwaptionEngine = ql.Gaussian1dFloatFloatSwaptionEngine(gsr, 64, 7.0, True, False, ql.QuoteHandle(), ytsOis, True)

In [83]:
swaption4.setPricingEngine(floatSwaptionEngine)

In [84]:
reversionQuote=ql.QuoteHandle(ql.SimpleQuote(reversion))

In [85]:
leg0 = underlying4.leg(0)
leg1 = underlying4.leg(1)

In [86]:
cmsPricer = ql.LinearTsrPricer(swaptionVolHandle, reversionQuote)

In [87]:
ql.setCouponPricer(leg0,cmsPricer)

In [88]:
iborPricer = ql.BlackIborCouponPricer()

In [89]:
ql.setCouponPricer(leg1,iborPricer)

In [90]:
swapPricer = ql.DiscountingSwapEngine(ytsOis)

In [91]:
underlying4.setPricingEngine(swapPricer)

In [92]:
npv5 = underlying4.NPV()

In [93]:
print("Underlying CMS Swap NPV = %.6f" % npv5)

Underlying CMS Swap NPV = 0.004447


In [94]:
print("       CMS     Leg  NPV = %.6f" % underlying4.legNPV(0))

       CMS     Leg  NPV = -0.231736


In [95]:
print("       Euribor Leg  NPV = %.6f" % underlying4.legNPV(1))

       Euribor Leg  NPV = 0.236183


In [96]:
basket = swaption4.calibrationBasket(swapBase, swaptionVol, "Naive");

In [97]:
for helper in basket:
    helper.setPricingEngine(swaptionEngine)

In [98]:
gsr.calibrateVolatilitiesIterative(basket, method, ec)

In [99]:
printBasket(basket)


Expiry              Maturity            Nominal             Rate          Pay/Rec     Market ivol   
April 30th, 2015    May 6th, 2024       1.00                0.025307      0           0.200000      
May 3rd, 2016       May 6th, 2024       1.00                0.025300      0           0.200000      
May 3rd, 2017       May 6th, 2024       1.00                0.025303      0           0.200000      
May 3rd, 2018       May 6th, 2024       1.00                0.025306      0           0.200000      
May 2nd, 2019       May 6th, 2024       1.00                0.025311      0           0.200000      
April 30th, 2020    May 6th, 2024       1.00                0.025300      0           0.200000      
May 3rd, 2021       May 6th, 2024       1.00                0.025306      0           0.200000      
May 3rd, 2022       May 6th, 2024       1.00                0.025318      0           0.200000      
May 3rd, 2023       May 6th, 2024       1.00                0.025353      0           0.20

In [100]:
printModelCalibration(basket, gsr.volatility())


Expiry              Model sigma   Model price         Market price        Model ivol    Market ivol   
April 30th, 2015    0.005178      0.016111            0.016111            0.200000      0.200000      
May 3rd, 2016       0.005156      0.020062            0.020062            0.200000      0.200000      
May 3rd, 2017       0.005149      0.021229            0.021229            0.200000      0.200000      
May 3rd, 2018       0.005129      0.020738            0.020738            0.200000      0.200000      
May 2nd, 2019       0.005132      0.019096            0.019096            0.200000      0.200000      
April 30th, 2020    0.005074      0.016537            0.016537            0.200000      0.200000      
May 3rd, 2021       0.005091      0.013253            0.013253            0.200000      0.200000      
May 3rd, 2022       0.005097      0.009342            0.009342            0.200000      0.200000      
May 3rd, 2023       0.005001      0.004910            0.004910          

In [101]:
npv6 = swaption4.NPV()

In [102]:
print("Float swaption NPV (GSR) = %.6f" % npv6)

Float swaption NPV (GSR) = 0.004291


In [106]:
print("Float swap NPV (GSR) = %.6f" % swaption4.underlyingValue())

Float swap NPV (GSR) = 0.005250
